In [2]:
import sympy as sp

# Define symbols
EI, r, h = sp.symbols('EI r h')
xi = sp.symbols('xi')

# Hermite shape functions on [0,1]
H1 = 1 - 3*xi**2 + 2*xi**3
H2 = h * (xi - 2*xi**2 + xi**3)
H3 = 3*xi**2 - 2*xi**3
H4 = h * (-xi**2 + xi**3)
H = [H1, H2, H3, H4]

# Derivatives wrt s: d/ds = (1/h) d/dxi
dH = [sp.diff(H_i, xi) / h for H_i in H]
# Second derivatives wrt s: d2/ds2 = (1/h^2) d2/dxi2
ddH = [sp.diff(H_i, xi, 2) / h**2 for H_i in H]

# Local DOF symbols: x and y components
u1, u2, u3, u4 = sp.symbols('u1 u2 u3 u4')
v1, v2, v3, v4 = sp.symbols('v1 v2 v3 v4')
lam1, lam2 = sp.symbols('lam1 lam2')

# Build displacement fields
x_expr = sum(H[i]*[u1, u2, u3, u4][i] for i in range(4))
xp_expr = sum(dH[i]*[u1, u2, u3, u4][i] for i in range(4))
xpp_expr = sum(ddH[i]*[u1, u2, u3, u4][i] for i in range(4))

y_expr = sum(H[i]*[v1, v2, v3, v4][i] for i in range(4))
yp_expr = sum(dH[i]*[v1, v2, v3, v4][i] for i in range(4))
ypp_expr = sum(ddH[i]*[v1, v2, v3, v4][i] for i in range(4))

# Lambda interpolation on two nodes: linear shape
M1 = 1 - xi
M2 = xi
lam_expr = M1*lam1 + M2*lam2

# Define inextensibility residual term S = xp^2 + yp^2 - 1
S_expr = xp_expr**2 + yp_expr**2 - 1

# Local residual contributions R_loc_x and R_loc_y
R_loc_x = [
    sp.integrate(EI*xpp_expr*ddH[a] + 2*(lam_expr + r*S_expr)*xp_expr*dH[a], (xi, 0, 1))*h
    for a in range(4)
]
R_loc_y = [
    sp.integrate(EI*ypp_expr*ddH[a] + 2*(lam_expr + r*S_expr)*yp_expr*dH[a], (xi, 0, 1))*h
    for a in range(4)
]

# Full local residual vector
R_loc = sp.Matrix(R_loc_x + R_loc_y)

# Define all DOFs vector
dofs = [u1, u2, u3, u4, v1, v2, v3, v4, lam1, lam2]

# Compute the Jacobian: derivative of residual wrt all DOFs
J_loc = sp.simplify(R_loc.jacobian(dofs))

# Display the symbolic local Jacobian
J_loc


Matrix([
[ 6*(70*EI + 7*h**2*lam1 + 7*h**2*lam2 + 3*h**2*r*u2**2 + 3*h**2*r*u4**2 + h**2*r*v2**2 + h**2*r*v4**2 - 14*h**2*r + 18*h*r*u1*u2 + 18*h*r*u1*u4 - 18*h*r*u2*u3 - 18*h*r*u3*u4 + 6*h*r*v1*v2 + 6*h*r*v1*v4 - 6*h*r*v2*v3 - 6*h*r*v3*v4 + 72*r*u1**2 - 144*r*u1*u3 + 72*r*u3**2 + 24*r*v1**2 - 48*r*v1*v3 + 24*r*v3**2)/(35*h**3),                                                                   (420*EI + 14*h**2*lam2 - 3*h**2*r*u2**2 + 6*h**2*r*u2*u4 + 3*h**2*r*u4**2 - h**2*r*v2**2 + 2*h**2*r*v2*v4 + h**2*r*v4**2 - 14*h**2*r + 72*h*r*u1*u2 - 72*h*r*u2*u3 + 24*h*r*v1*v2 - 24*h*r*v2*v3 + 108*r*u1**2 - 216*r*u1*u3 + 108*r*u3**2 + 36*r*v1**2 - 72*r*v1*v3 + 36*r*v3**2)/(70*h**2), 6*(-70*EI - 7*h**2*lam1 - 7*h**2*lam2 - 3*h**2*r*u2**2 - 3*h**2*r*u4**2 - h**2*r*v2**2 - h**2*r*v4**2 + 14*h**2*r - 18*h*r*u1*u2 - 18*h*r*u1*u4 + 18*h*r*u2*u3 + 18*h*r*u3*u4 - 6*h*r*v1*v2 - 6*h*r*v1*v4 + 6*h*r*v2*v3 + 6*h*r*v3*v4 - 72*r*u1**2 + 144*r*u1*u3 - 72*r*u3**2 - 24*r*v1**2 + 48*r*v1*v3 - 24*r*v3**2)/(35*h**